# Colab-MST

Original repo: [ewrfcas/MST_inpainting](https://github.com/ewrfcas/MST_inpainting)

My fork: [styler00dollar/Colab-MST](https://github.com/styler00dollar/Colab-MST)

In [ ]:
!nvidia-smi

In [ ]:
#@title install
%cd /content/
!git clone https://github.com/ewrfcas/MST_inpainting
%cd /content/MST_inpainting
!mkdir -p /content/MST_inpainting/check_points/LSM_HAWP/
%cd ./check_points/LSM_HAWP/
!gdown --id 1ZE7pWCI-sTmNZvJOBy2DvL3m1qbybWcH
!pip install yacs
%cd /content/MST_inpainting/
!gdown --id 1wTNmBxaqDTmp0bA2okN_oW60JPIGPR80
!gdown --id 1ZcNsq48aZzOmJ2v-nrFh9ugn4jCUaUkL

In [ ]:
#@title create masks from green marked images
import cv2
import glob
import os
import numpy as np
from tqdm import tqdm
rootdir = "/content/input"
destdir = "/content/mask"
files = glob.glob(rootdir + '/**/*.png', recursive=True)

for f in tqdm(files):
  image = cv2.imread(f)
  mask = (1-np.all(image == [0,255,0], axis=-1).astype(int))*255
  # inverting color
  mask = 255-mask 
  # dilate mask
  kernel = np.ones((7,7), np.uint8)
  mask = cv2.dilate(mask.astype('uint8'), kernel, iterations=1)
  cv2.imwrite(os.path.join(destdir, os.path.basename(f)), mask)

In [ ]:
# test single image
%cd /content/MST_inpainting
!python test_single.py --gpu_id 0 \
                      --PATH /content/MST_inpainting \
                      --image_path /content/input/test.png \
                      --mask_path /content/mask/test.png

In [ ]:
#@title test_folder.py
%%writefile test_folder.py
import argparse
import os
import random

import numpy as np
import torch
import glob
from src.model_inference import MST
from utils.utils import Config

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--image_path', type=str, required=True, help='image path')
    parser.add_argument('--mask_path', type=str, required=True, help='mask path')
    parser.add_argument('--gpu_id', type=int, default=0, help='gpu id')
    parser.add_argument('--PATH', type=str, default='./check_points/MST_P2M',
                        help='MST_P2M:Man-made Places2, MST_P2C:Comprehensive Places2, '
                             'MST_shanghaitech:all man-made scenes')
    parser.add_argument('--valid_th', type=float, default=0.85)
    parser.add_argument('--mask_th', type=float, default=0.8)
    parser.add_argument('--not_obj_remove', action='store_true', default=False)
    parser.add_argument('--config_path', type=str, default='./config.yml', help='config path')

    args = parser.parse_args()
    os.environ['CUDA_VISIBLE_DEVICES'] = str(args.gpu_id)

    # load config file
    config = Config(args.config_path)

    # test mode
    config.PATH = args.PATH
    config.valid_th = args.valid_th
    config.mask_th = args.mask_th

    # init device
    if torch.cuda.is_available():
        config.DEVICE = torch.device("cuda")
        torch.backends.cudnn.benchmark = True
    else:
        config.DEVICE = torch.device("cpu")

    print('DEVICE:', config.DEVICE)

    # initialize random seed
    torch.manual_seed(config.SEED)
    torch.cuda.manual_seed_all(config.SEED)
    np.random.seed(config.SEED)
    random.seed(config.SEED)

    concat_mask = True if 'P2C' in args.PATH else False
    model = MST(config, concat_mask)
    model.load()

    files = glob.glob(args.image_path + '/**/*.png', recursive=True)

    for f in files:
      model.inference(f, os.path.join(args.mask_path, os.path.basename(f)), config.valid_th, config.mask_th,
                      not_obj_remove=args.not_obj_remove)

In [ ]:
# test folder
%cd /content/MST_inpainting
!python test_folder.py --gpu_id 0 \
                      --PATH /content/MST_inpainting \
                      --image_path /content/input/ \
                      --mask_path /content/mask/